In [1]:
import pandas as pd
from glob import glob
import os
# --------------------------------------------------------------
# Read single CSV file
# --------------------------------------------------------------
# gia tốc kế 
print(os.getcwd())
single_file_acc = pd.read_csv("../../data/raw/MetaMotion/A-bench-heavy2-rpe8_MetaWear_2019-01-11T16.10.08.270_C42732BE255C_Accelerometer_12.500Hz_1.4.4.csv")
# con quay hồi chuyển 
single_file_gyr = pd.read_csv("../../data/raw/MetaMotion/A-bench-heavy2-rpe8_MetaWear_2019-01-11T16.10.08.270_C42732BE255C_Gyroscope_25.000Hz_1.4.4.csv")


# --------------------------------------------------------------
# List all data in data/raw/MetaMotion
# --------------------------------------------------------------
# tạo các khối xây dựng riêng lẻ sau đó sẽ kết hợp chúng với nhau 
files = glob("../../data/raw/MetaMotion/*.csv")
len(files)


# --------------------------------------------------------------
# Extract features from filename
# --------------------------------------------------------------
data_path = "../../data/raw/MetaMotion/"
f = files[1]
f = os.path.normpath(f)
data_path = os.path.normpath(data_path)

participant = f.split("-")[0].replace(data_path + os.sep, "")
label = f.split("-")[1]
category = f.split("-")[2].rstrip("123").rstrip("_MetaWear_2019")

df = pd.read_csv(f)
df["participant"] = participant
df["label"] = label
df["category"] = category
# --------------------------------------------------------------
# Read all files
# --------------------------------------------------------------
acc_df = pd.DataFrame()
gyr_df = pd.DataFrame()

acc_set = 1
gyr_set = 1

for f in files:
    f = os.path.normpath(f)
    data_path = os.path.normpath(data_path)

    participant = f.split("-")[0].replace(data_path + os.sep, "")
    label = f.split("-")[1]
    category = f.split("-")[2].rstrip("123").rstrip("_MetaWear_2019")

    df = pd.read_csv(f)
    df["participant"] = participant
    df["label"] = label
    df["category"] = category

    # chia khung dữ liệu gia tốc kế (Accelerometer) và con quay hồi chuyển (Gyroscope)
    # khoảng 23.000 bản ghi cho gia tốc kế và 47.000 bản ghi cho dữ liệu con quay hồi chuyển 
    # sự khác biệt là con quay hồi chuyển đo ở tần số cao hơn 
    if "Accelerometer" in f:
        df["set"] = acc_set
        acc_set+=1
        acc_df = pd.concat([acc_df,df])

    if "Gyroscope" in f:
        df["set"] = gyr_set
        gyr_set+=1
        gyr_df = pd.concat([gyr_df,df])

# --------------------------------------------------------------
# Working with datetimes
# --------------------------------------------------------------
# epoch biến thời gian 
# time : Unix time là cách biểu diễn ngày và giờ được sử dụng rộng rãi trong máy tính. Nó đo thời gian bằng số giây (hoặc mili giây/nano giây) đã trôi qua kể từ 00:00:00 UTC ngày 1 tháng 1 năm 1970, thời điểm bắt đầu kỷ nguyên Unix.
acc_df.info()

pd.to_datetime(df["epoch (ms)"],unit="ms")

acc_df.index = pd.to_datetime(acc_df["epoch (ms)"],unit="ms")
gyr_df.index = pd.to_datetime(gyr_df["epoch (ms)"],unit="ms")

del acc_df["epoch (ms)"]
del acc_df["time (01:00)"]
del acc_df["elapsed (s)"]

del gyr_df["epoch (ms)"]
del gyr_df["time (01:00)"]
del gyr_df["elapsed (s)"]
# --------------------------------------------------------------
# Turn into function
# --------------------------------------------------------------
files = glob("../../data/raw/MetaMotion/*.csv")
def read_data_from_files(files):
    acc_df = pd.DataFrame()
    gyr_df = pd.DataFrame()

    acc_set = 1
    gyr_set = 1

    for f in files:
        f = os.path.normpath(f)
        data_path = os.path.normpath(data_path)

        participant = f.split("-")[0].replace(data_path + os.sep, "")
        label = f.split("-")[1]
        category = f.split("-")[2].rstrip("123").rstrip("_MetaWear_2019")

        df = pd.read_csv(f)
        df["participant"] = participant
        df["label"] = label
        df["category"] = category

        # chia khung dữ liệu gia tốc kế (Accelerometer) và con quay hồi chuyển (Gyroscope)
        # khoảng 23.000 bản ghi cho gia tốc kế và 47.000 bản ghi cho dữ liệu con quay hồi chuyển 
        # sự khác biệt là con quay hồi chuyển đo ở tần số cao hơn 
        if "Accelerometer" in f:
            df["set"] = acc_set
            acc_set+=1
            acc_df = pd.concat([acc_df,df])

        if "Gyroscope" in f:
            df["set"] = gyr_set
            gyr_set+=1
            gyr_df = pd.concat([gyr_df,df])

        acc_df.index = pd.to_datetime(acc_df["epoch (ms)"],unit="ms")
        gyr_df.index = pd.to_datetime(gyr_df["epoch (ms)"],unit="ms")

        del acc_df["epoch (ms)"]
        del acc_df["time (01:00)"]
        del acc_df["elapsed (s)"]

        del gyr_df["epoch (ms)"]
        del gyr_df["time (01:00)"]
        del gyr_df["elapsed (s)"]

        return acc_df,gyr_df
    
    acc_df,gyr_df = read_data_from_files(files)
# --------------------------------------------------------------
# Merging datasets
# --------------------------------------------------------------
data_merge = pd.concat([acc_df.iloc[:,:3],gyr_df],axis=1)
data_merge.columns = [
    "acc_x",
    "acc_y",
    "acc_z",
    "gyr_x",
    "gyr_y",
    "gyr_z",
    "participant",
    "label",
    "category",
    "set",
]
# --------------------------------------------------------------
# Resample data (frequency conversion)
# --------------------------------------------------------------

# Accelerometer:    12.500HZ
# Gyroscope:        25.000Hz
# 1/25 độ đo của gia tốc kế
# 1/12.5 độ đo của con quay hồi chuyển 
sampling = {
    "acc_x" : "mean", 
    "acc_y" : "mean", 
    "acc_z" : "mean", 
    "gyr_x" : "mean", 
    "gyr_y" : "mean", 
    "gyr_z" : "mean", 
    "participant" : "last", 
    "label" : "last",
    "category" : "last", 
    "set" : "last"
}

data_merge[:1000].resample(rule="200ms").apply(sampling)
# split by day 
days = [g for n , g in data_merge.groupby(pd.Grouper(freq="D"))]
# danh sách lặp lại nhiều ngày từ ngày 11 đến 20 
# dropna xóa những hàng nào có giá trị là null 
data_resampled = pd.concat([df.resample(rule="200ms").apply(sampling).dropna() for df in days])
data_resampled["set"] = data_resampled["set"].astype("int")
# data_resampled.info()

# --------------------------------------------------------------
# Export dataset
# --------------------------------------------------------------
data_resampled.to_pickle("../../data/interim/01_data_processed.pkl")
pd.set_option("display.max_columns", None) 
pd.set_option("display.width", None)  
pd.set_option("display.expand_frame_repr", False) 
print(data_resampled)


d:\tracking-barbell-exercises\src\data
<class 'pandas.core.frame.DataFrame'>
Int64Index: 23578 entries, 0 to 241
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   epoch (ms)    23578 non-null  int64  
 1   time (01:00)  23578 non-null  object 
 2   elapsed (s)   23578 non-null  float64
 3   x-axis (g)    23578 non-null  float64
 4   y-axis (g)    23578 non-null  float64
 5   z-axis (g)    23578 non-null  float64
 6   participant   23578 non-null  object 
 7   label         23578 non-null  object 
 8   category      23578 non-null  object 
 9   set           23578 non-null  int64  
dtypes: float64(4), int64(2), object(4)
memory usage: 2.0+ MB
                            acc_x     acc_y     acc_z   gyr_x   gyr_y   gyr_z participant  label category  set
epoch (ms)                                                                                                    
2019-01-11 15:08:05.200  0.013500  0.977000 -0.071000 